In [ ]:
import os
import sys
import re
import pandas as pd
import csv
from functools import partial
sys.path.append("/home/kai/muku_esg/ESG-BERT/")
from allgrammer import grammer
from pdfminer.high_level import extract_text
%cd /home/kai/muku_esg/ESG-BERT/

In [ ]:
def ascii(text):
    text = re.sub(r'[^\x00-\x7F]', '', text)
    text = re.sub(r'[\x0C]', r"\n\n", text)
    return text

In [ ]:
folder = 'taiwan_pdf'
csv_folder = 'predict_taiwan'
word_cnt = 'eight_new'

In [ ]:
# folder = 'american_pdf'
# csv_folder = 'predict_american'
# word_cnt = 'eight_new'

In [ ]:
def preprocess(x, folder, file_name):
    x = x.replace('\n', ' ')
    x = re.sub(r'[!\"#$%&\'()*\+,-.\/:;<=>?@\[\\\]^_`{|}~]', '', x)
    x = x.replace('\t', ' ')
    x = x.strip()
    if len(x.split()) < 8:
        return None
    else:
        return grammer(x, folder, file_name)

In [ ]:
file_names = ['15_msi_1_97.pdf',
   '16_gigabyte_1_74.pdf', '17_nanya_1_154.pdf', '18_novatek_1_140.pdf', '19_mediatek_1_97.pdf', '20_asia_1_119.pdf', 
   '21_cathay_1_52.pdf', '22_cht_1_79.pdf', '23_fubon_1_89.pdf', '24_mega_1_68.pdf']
for file_name in file_names:
    file_name = file_name.strip('.pdf')
    os.chdir('/home/kai/muku_esg/ESG-BERT')
    text = extract_text(f"./{folder}/{file_name}.pdf")
    text = ascii(text)
    text = text.strip('\"')
    paragraph = re.split(r'\n\s*\n', text)
    constant_preprocess = partial(preprocess, folder=folder, file_name=file_name)
    paragraph = map(constant_preprocess, paragraph)
    paragraph = list(paragraph)
    y = []
    for i in range(len(paragraph)):
        if paragraph[i] != None:
            x = paragraph[i].split()
            fifthlist = x[:300]
            fifthstr = ' '.join(fifthlist)
            y.append(fifthstr)
    paragraph = y
    #   predict
    os.chdir('./predict_american')
    with open("./output.txt", "w") as file:
        file.truncate()
    with open("./paragraph.txt", "w") as file:
        file.truncate()
    for i in range(len(paragraph)):
        # Open a file in write mode
        with open("./paragraph.txt", "w") as f:
            # Write the strings to the file
            f.writelines(paragraph[i])
        os.system(f'curl -X POST http://127.0.0.1:8080/predictions/bert -T paragraph.txt >> output.txt')
        os.system(f'echo >> output.txt')
    #   read output label and create csv
    with open('./output.txt', 'r') as f:
        labels = f.readlines()
        labels = [x.strip('\n') for x in labels]
    ans = pd.DataFrame({'paragraph': paragraph, 'predict_label': labels})
    ans.to_csv(f'../{csv_folder}/{word_cnt}/{file_name}.csv', sep=',', index=False, encoding='utf-8')

In [4]:
# Append the predict_label to csv
os.chdir('/home/kai/muku_esg/ESG-BERT/predict_taiwan/five_word')
file_name = '08_witron_1_93'
with open(f'./{file_name}.csv', 'r') as f:
    # Create a CSV reader object
    reader = csv.reader(f)
    paragraphs = []
    labels = []
    for row in reader:
        if row[0][:27] == '2021 Wistron ITS ESG Report' :
            pass
        else:
            paragraphs.append(row[0])
            labels.append(row[1])
    ans = pd.DataFrame({'paragraph': paragraphs, 'predict_label': labels})
    ans.to_csv(f'../five_new/{file_name}.csv', sep=',', index=False, encoding='utf-8')